# Summary

<div class="alert alert-block alert-warning">
This notebook provides a reference procedure of forecasting default rates for Fannie Mae. <br>
<br>
We first fit logistic regression, regression tree, and xgoost to the training set, and predict the probability of default for the loans in the testing set. The accuracy is measured by the Brier Skill Score (higher BSS indicates more accurate predictions). Next, we provide an example of stacking. 
<br>
The accuracy of the four methods are listed bellow: 

| Method | Brier Skill Score      
| :-: | :-:
|logistic regression|  0.0499
|regression tree|  0.0495
|xgboost | 0.0527
|weighted average ensemble | $\bf{0.0531}$
|stacking model-RT | 0.0515
|stacking model_XGBOOST| 0.0526



## Table of Content <br>
[Read in the Training and Testing Data](#Read-in-the-Training-and-Testing-Data)<br>
[Function for Calculating the Brier Skill Score](#Function-for-Calculating-the-Brier-Skill-Score)<br>
[Logistic Regression](#Logistic-Regression)<br>
[Regression Tree](#Regression-Tree)<br>
[XGBOOST](#XGBOOST)<br>
[Ensemble Model --- Weighted Average](#Ensemble-Model-----Weighted-Average)<br>
[Ensemble Model --- Stacking](#Ensemble-Model-----Stacking)<br>

Load the packaes and set up for parallel processing

In [27]:
# You need to run this line if you are using Jupyter server
.libPaths("/usr/local/lib/R/site-library") 

In [28]:
library(parallel)
library(doSNOW)
library(foreach)
library(Matrix)
library(rpart)
library(randomForest)
library(xgboost)
# Set up your multiple cores as separate workers and then make them a cluster.
workers <- detectCores()
workers
cluster <- makeCluster(workers, type = "SOCK")
registerDoSNOW(cluster)

[1] 4

## Read in the Training and Testing Data

<div class="alert alert-block alert-warning">The .rda files allow a user to save their R data structures such as vectors, matrices, and data frames. The file is automatically compressed. 

In [29]:
# Read in the data
training <- load("fannieMae_training.Rda") 
testing <- load("fannieMae_testing.Rda") 

In [30]:
# The data saved in the Rda file already has a variable name.
# See the name of the data table saved in the rda file.
print(training)
print(testing)

[1] "train.df"
[1] "test.df"


## Function for Calculating the Brier Skill Score

<div class="alert alert-block alert-warning"> A Brier score is a way to verify the accuracy of a probability forecast. The score can only be used for binary outcomes, where there are only two possible events. The formula to calculate the Brier score is given by <br>
<br>
$BS = mean(f_t - o_t)^2$ <br>
<br>
where $f_t$ is the forecast probability, and $o_t$ is the outcome.

A Brier Skill Score (BSS) shows the relative skill of your probability forecast over that of a reference method. A BSS of 1.0 indicates a perfect forecast, while a BSS of 0.0 should indicate the skill of the reference forecast. Less than zero indicates worse than this reference forecast performance. The higher this score is the better. The formula of calculating the BSS is <br>
<br>
$BSS = 1 - \frac{BS}{BS^{ref}}$

(See http://www.statisticshowto.com/brier-score/ for a detailed introduction of Brier Skill Score) <br>
<br>
In this study we use a naive forecast as the reference method. The naive forecast is the proportion of default loans in the training set.

In [31]:
# Write a function to calculate the average Brier score.  
brier.score <- function(pred, real) {
 return(mean((pred - real)^2))   
}

# Establish a reference brier score for a naive forecast
# Naive forecast: use the propotion of default loans in the traning set as the prediction 
# for all the loans in the testing set.
train.default.rate <- mean(train.df$DEFAULT_FLAG)
test.realizations <- test.df$DEFAULT_FLAG
naive.pred <- rep(train.default.rate, length(test.realizations))
brier.ref <- brier.score(naive.pred, test.realizations)

# Write a function to calculate the Brier skill score
skill.score <- function(pred, real, brier.ref) {
    brier.score <- brier.score(pred, real) # calculate the Brier score for your predictions.
    return(1 - brier.score/ brier.ref)
}

## Logistic Regression

### Fit the model

<div class="alert alert-block alert-warning"> Select following variables as predictors in my model:<br>

OLTV: ORIGINAL LOAN-TO-VALUE. Calculated by dividing the original loan amount by the value of the property. <br>
<br>
DTI: Debt-to-Income. Calculated by dividing the borrower’s total monthly obligations by his or her stable monthly income. <br>
<br>
CSCORE_B: Borrower Credit Score. 

In [32]:
# print out column names
print(colnames(train.df))

 [1] "LOAN_ID"         "ORIG_CHN"        "Seller.Name"     "ORIG_RT"        
 [5] "ORIG_AMT"        "ORIG_TRM"        "ORIG_DTE"        "FRST_DTE"       
 [9] "OLTV"            "OCLTV"           "NUM_BO"          "DTI"            
[13] "CSCORE_B"        "FTHB_FLG"        "PURPOSE"         "PROP_TYP"       
[17] "NUM_UNIT"        "OCC_STAT"        "STATE"           "ZIP_3"          
[21] "RELOCATION_FLG"  "Monthly.Rpt.Prd" "Loan.Age"        "Maturity.Date"  
[25] "MSA"             "DEFAULT_FLAG"   


In [33]:
# Select OLTV, DTI, CSCORE_B as predictors. DEFAULT_FLAG is the response variable. 
train.df.select <- train.df[,c(9, 12, 13, 26)]
test.df.select <- test.df[,c(9, 12, 13,  26)]

In [34]:
logit.reg <- glm(DEFAULT_FLAG~., data = train.df.select, family = "binomial")

### Predict and test accuracy in the testing set

In [35]:
pred.logit.reg <- predict(logit.reg, test.df.select, type = "response")

# Calculate the Brier Skill Score
skill.score(pred.logit.reg, test.df.select$DEFAULT_FLAG, brier.ref)

[1] 0.04874738

<div class="alert alert-block alert-warning"> We can also try to add interactions between variables.

In [36]:
logit.reg <- glm(DEFAULT_FLAG ~ . + .^2 , data = train.df.select, family = "binomial")
summary(logit.reg)


Call:
glm(formula = DEFAULT_FLAG ~ . + .^2, family = "binomial", data = train.df.select)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.4407  -0.3882  -0.2645  -0.1569   3.9991  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)    1.085e+01  6.283e-01  17.269  < 2e-16 ***
OLTV          -4.543e-02  7.493e-03  -6.063 1.34e-09 ***
DTI           -5.631e-02  8.502e-03  -6.623 3.51e-11 ***
CSCORE_B      -2.536e-02  8.908e-04 -28.469  < 2e-16 ***
OLTV:DTI       4.712e-05  5.164e-05   0.913    0.361    
OLTV:CSCORE_B  1.297e-04  1.027e-05  12.629  < 2e-16 ***
DTI:CSCORE_B   1.068e-04  1.166e-05   9.152  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 144580  on 333289  degrees of freedom
Residual deviance: 128357  on 333283  degrees of freedom
AIC: 128371

Number of Fisher Scoring iterations: 7


In [37]:
pred.logit.reg <- predict(logit.reg, test.df.select, type = 'response')
skill.score(pred.logit.reg, test.df.select$DEFAULT_FLAG, brier.ref)

[1] 0.04994659

## Regression Tree

### Fit the model

In [38]:
library(rpart)
reg.tree <- rpart(DEFAULT_FLAG ~ ., data = train.df.select, control = rpart.control(cp = 0.0001 ) )

### Print out feature importance

In [39]:
reg.tree$variable.importance

OLTV CSCORE_B      DTI 
443.8604 387.9229 103.1858

### Predict and test accuracy in the testing set

In [40]:
reg.tree.pred <- predict(reg.tree, test.df.select)

# score the regression tree model
# skill.score(reg.tree.pred, valid.df[, length(selected) + 1])
skill.score(reg.tree.pred, test.df.select$DEFAULT_FLAG, brier.ref)

[1] 0.04954727

## XGBOOST

### Fit the model

In [41]:
xtrain.xgb <- sparse.model.matrix(~ 0 + ., data = train.df.select[,-4])
ytrain.xgb <- as.vector(train.df.select$DEFAULT_FLAG)

xtest.xgb <- sparse.model.matrix(~ 0 + ., data = test.df.select[,-4])
ytest.xgb <- as.vector(test.df.select$DEFAULT_FLAG)

In [42]:
# Generate the xgboost model, also taking advantage of parallel processing
xgb.trees <- xgboost(xtrain.xgb, ytrain.xgb, 
                     max.depth = 3, 
                     nthread = workers, 
                     nround = 200, 
                     objective = "binary:logistic",
                     verbose = 0)

# Tune the model above by increasing/decreasing the depth of each tree 
# and/or the number of trees (nround).

### Print out feature importance

In [43]:
# Gain indicates the contribution of each feature. 
# The higher the percentage, the greater the contribution.

xgb.importance(colnames(xtrain.xgb), model = xgb.trees)  

Feature,Gain,Cover,Frequency
OLTV,0.4361497,0.3480568,0.3055130
CSCORE_B,0.4309340,0.4893491,0.4218989
DTI,0.1329164,0.1625941,0.2725881


### Predict and test accuracy in the testing set

In [44]:
# Score the model's forecast

xgb.trees.pred <- predict(xgb.trees, xtest.xgb)
skill.score(xgb.trees.pred, ytest.xgb, brier.ref)

[1] 0.0526532

## Ensemble Model --- Weighted Average

In [20]:
m1_weight <- seq(0.1,0.9,0.1)
m2_weight <- seq(0.1,0.9,0.1)

# set up a matrix to store the Brier skill scores
bss_matrix <- matrix(0,9,9)
for (i in 1:9) {
    for (j in 1:9){
        if (m2_weight[j]+ m1_weight[i] > 1) next
        ensemble_pred <- m1_weight[i]*pred.logit.reg + reg.tree.pred*m2_weight[j] + xgb.trees.pred*(1-m1_weight[i] - m2_weight[i])
        bss_matrix[i,j] <- skill.score(ensemble_pred, test.df.select$DEFAULT_FLAG, brier.ref)
    }
}

In [26]:
bss_matrix # print out the Brier skills score matrix
which(bss_matrix == max(bss_matrix), arr.ind = TRUE) # which element in the matrix corresponds to the highest score?
# According to the table, the weights put on the predictions from the logistic regression, regression tree and xgboost
# should be 0.2,0.2,and 0.6. The highest Brier Skill Score acheived by the weighted average ensemble is 0.0532

0.053058896,0.051718749,0.04811727,0.04225445,0.03413030,0.02374482,0.01109800,-0.003810151,-0.02097964
0.052201945,0.053155415,0.05184755,0.04827835,0.04244782,0.03435596,0.02400276,0.011388220,0.00000000
0.048709920,0.051957008,0.05294276,0.05166718,0.04813027,0.04233202,0.03427244,0.000000000,0.00000000
0.042582822,0.048123528,0.05140290,0.05242094,0.05117764,0.04767301,0.00000000,0.000000000,0.00000000
0.033820651,0.041654974,0.04722796,0.05053962,0.05158994,0.00000000,0.00000000,0.000000000,0.00000000
0.022423406,0.032551347,0.04041795,0.04602323,0.00000000,0.00000000,0.00000000,0.000000000,0.00000000
0.008391088,0.020812646,0.03097287,0.00000000,0.00000000,0.00000000,0.00000000,0.000000000,0.00000000
-0.008276303,0.006438872,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.000000000,0.00000000
-0.027578768,0.000000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.000000000,0.00000000


row,col
2,2


## Ensemble Model --- Stacking

<div class="alert alert-block alert-warning"> This section provides an example combining three base models: a logistic regression, a regression tree, and a xgboost model. We try to use a regression tree and a xgboost model as the stacker model respectively. The procedure can be summarised as follows:<br>

1. Split the training set into two parts, a smaller training set and a validation set.
2. Train the base models (M1, M2 and M3) by using the smaller training set, and generate predictions from these three models in the validation set.
3. Fit a regression tree (or an XGBOOST model) to the predictions generated in Step 2. The dependent variable is DEFAULT_FLAG in the training set and the independent variables are the predictions obtained in Step 2.
4. Re-train the base models using the entire training set and generate predictions in the testing set.

###  Step 1. Split the training set into a smaller training set and a validation set.

Here is an example of a 75/25 split.

In [46]:
(nrowTrain <- nrow(train.df.select))
(nrowSmallTrain <- round(nrowTrain*0.75))
(nrowvalid <- nrowTrain - nrowSmallTrain)

set.seed(201)
# generate row numbers of the training set.
rowIndicesSmallTrain <- sample(1:nrowTrain, size = nrowSmallTrain, replace = FALSE) 
smalltrain.df <- train.df.select[rowIndicesSmallTrain, ]
valid.df <- train.df.select[-rowIndicesSmallTrain, ]

[1] 333290

[1] 249968

[1] 83322

### Step 2. Train the base models in the small training set and generate predictions in the validation set

In [47]:
# Logistic regression
M1 <- glm(DEFAULT_FLAG ~  . + (.)^2, data = smalltrain.df, family = "binomial")
M1.predict <- predict.glm(M1, valid.df, type = "response")

In [48]:
# Regression tree
M2 <- rpart(DEFAULT_FLAG ~ ., 
                  data = smalltrain.df, 
                  control = rpart.control(cp = 0.0001), 
                  method = "anova")
M2.predict <-  predict(M2, valid.df)

In [54]:
# XGBOOST
smallxtrain.xgb <- sparse.model.matrix(~ 0 + ., data = smalltrain.df[,-4])
smallytrain.xgb <- as.vector(smalltrain.df$DEFAULT_FLAG)

xvalid.xgb <- sparse.model.matrix(~ 0 + ., data = valid.df[,-4])
yvalid.xgb <- as.vector(valid.df$DEFAULT_FLAG)

M3 <- xgboost(smallxtrain.xgb, smallytrain.xgb, 
                     max.depth = 3, 
                     nthread = workers, 
                     nround = 200, 
                     objective = "binary:logistic",
                     verbose = 0)


M3_predict <- predict(M3, xvalid.xgb)

### Step 4. Fit a stacker model to the predictions generated in Step 2. 

Create a data frame as the input for the stacking model.

In [144]:
stacker.df <- data.frame(DEFAULT_FLAG = valid.df$DEFAULT_FLAG, 
                         M1.predict = M1.predict, 
                         M2.predict = M2.predict,
                         M3.predict = M3_predict)

head(stacker.df)

DEFAULT_FLAG,M1.predict,M2.predict,M3.predict
0,0.01121448,0.01419676,0.01032643
0,0.03839097,0.03869611,0.03180914
0,0.11456575,0.14865616,0.12453716
1,0.05028825,0.07231588,0.09881273
0,0.17325774,0.09760226,0.11512689
0,0.09776487,0.09623558,0.10042360


Fit a regression tree to the predictions as stacker model 1

In [145]:
stackerModel_1 <- rpart(DEFAULT_FLAG ~ ., 
                  data = stacker.df, 
                  control = rpart.control(cp = 0.0004), # Different settings of cp have been examined. Here 0.0004 gives the best accuracy in the test set.
                  method = "anova")

Fit an XGBOOST model to the predictions as stacker model 2

In [146]:
stacker.df.xgb <- model.matrix(~ 0 + ., data = stacker.df[,-1])
stacker.y <- as.vector(stacker.df[,1])
stackerModel_2 <- xgboost(stacker.df.xgb, stacker.y,
                     max.depth = 3, 
                     nthread = workers, 
                     nround = 20, 
                     objective = "binary:logistic",
                     verbose = 0)
# Different settings of max.depth and nround have been examined. 
# max.depth = 3 and nround = 20 provide the highest Brier Skill Score in the test set.

### Step 5. Re-train the base models using the entire training set and generate predictions for the validation set

Re-train the base models to the entire training set (train.df.select).

In [57]:
M1.trainAll <- glm(DEFAULT_FLAG ~ . + .^2, 
                   data = train.df.select, family = "binomial")

M2.trainAll <- rpart(DEFAULT_FLAG ~ ., 
                  data = train.df.select, 
                  control = rpart.control(cp = 0.0001), 
                  method = "anova")

M3.trainAll <- xgboost(xtrain.xgb, ytrain.xgb, 
                     max.depth = 3, 
                     nthread = workers, 
                     nround = 200, 
                     objective = "binary:logistic",
                     verbose = 0)

Generate predictions for the testing set.

In [70]:
M1.predict.test <- predict(M1.trainAll, test.df.select, type = "response")
M2.predict.test <- predict(M2.trainAll, test.df.select)
M3.predict.test <- predict(M3.trainAll, xtest.xgb)

Use M1.predict.test, M2.predict.test and M3.predict.test as inputs to the stacker model.

In [147]:
predict.variables <- data.frame('M1.predict' = M1.predict.test, 
                                'M2.predict' = M2.predict.test,
                                'M3.predict' = M3.predict.test)

# Predict from stacker model 1 --- regression tree
stacker.predict.rt <- predict(stackerModel_1, predict.variables)

# Score the stacker model's prediction
skill.score(stacker.predict.rt, test.df.select$DEFAULT_FLAG, brier.ref) 

[1] 0.05152645

In [148]:
# Predict from stacker model 2 --- XGBOOST, and calculate the Brier Skill Score
predict.variables.xgb <- model.matrix(~ 0 + ., data = predict.variables)
stacker.predict.xgb <- predict(stackerModel_2, predict.variables.xgb)

# Score the stacker model's prediction
skill.score(stacker.predict.xgb, test.df.select$DEFAULT_FLAG, brier.ref) 

[1] 0.05255843